In [41]:
from langchain_community.document_loaders import DirectoryLoader, PDFPlumberLoader
from pathlib import Path
import os

# PDF 파일이 있는 디렉토리 경로
pdf_dir = "C:\\Users\\user\\Documents\\GitHub\\DACON-construction-accident-prevention\\data\\pdf"  # 원하는 디렉토리 경로로 변경

# DirectoryLoader를 사용하여 모든 PDF 파일 로드
loader = DirectoryLoader(pdf_dir, glob="*.pdf", loader_cls=PDFPlumberLoader)  
all_documents = loader.load()  # 전체 PDF 파일 로드

In [57]:
all_documents[0].metadata

{'source': 'C:\\Users\\user\\Documents\\GitHub\\DACON-construction-accident-prevention\\data\\pdf\\F.C.M 교량공사 안전보건작업 지침.pdf',
 'file_path': 'C:\\Users\\user\\Documents\\GitHub\\DACON-construction-accident-prevention\\data\\pdf\\F.C.M 교량공사 안전보건작업 지침.pdf',
 'page': 0,
 'total_pages': 24,
 'Producer': 'ezPDF Builder Supreme',
 'CreationDate': "D:20201223022000+09'00'",
 'ModDate': "D:20201223022000+09'00'"}

In [63]:
import re
from langchain_core.documents import Document

# 📌 특정 파일명이 2번째 등장하기 전까지 삭제하는 함수
def remove_text_before_second_filename(documents, filename):
    """
    로드된 Document 리스트에서 파일명이 2번째 등장하기 전까지의 내용을 삭제하는 함수.
    """
    base_filename = os.path.splitext(filename)[0]  # 파일명에서 확장자 제거 (예: 'sample.pdf' -> 'sample')

    # 모든 문서 내용을 하나로 이어붙임
    full_text = "\n".join([doc.page_content for doc in documents])

    # 파일명이 등장하는 모든 위치 찾기
    matches = [match.start() for match in re.finditer(rf"\b{re.escape(base_filename)}\b", full_text, re.IGNORECASE)]

    # 파일명이 2번 이상 등장하는 경우, 두 번째 등장 이후의 텍스트만 유지
    if len(matches) >= 2:
        cleaned_text = full_text[matches[1]:].strip()
    else:
        cleaned_text = full_text  # 파일명이 2번 이상 등장하지 않으면 원본 유지

    # 새로운 Document 객체로 반환
    return [Document(page_content=cleaned_text, metadata={"source": filename})]

# 📌 각 PDF 파일별로 처리
processed_documents = []
for pdf_file in sorted(Path(pdf_dir).glob("*.pdf")):
    # 해당 PDF에 해당하는 문서들 필터링
    pdf_documents = [doc for doc in all_documents if pdf_file.name in doc.metadata.get("source", "")]
    
    if pdf_documents:
        cleaned_docs = remove_text_before_second_filename(pdf_documents, pdf_file.name)
        processed_documents.extend(cleaned_docs)

# ✅ 결과 확인 (첫 3개만 출력)
for doc in processed_documents[:3]:
    print(f"📄 Source: {doc.metadata.get('source', 'Unknown')}")
    print(doc.page_content[:300])  # 앞부분 300자만 출력
    print("-" * 40)

📄 Source: F.C.M 교량공사 안전보건작업 지침.pdf
F.C.M 교량공사 안전보건작업 지침
1. 목 적
이 지침은 산업안전보건기준에 관한 규칙(이하 “안전보건규칙”이라 한다.) 제
42조~제49조(추락에 의한 위험방지), 제133조~제150조(크레인, 이동식 크레인),
제369조(교량작업), 제328조~제337조(거푸집동바리 등)의 규정에 의거 콘크리
트 구조물 교량공사(F.C.M공법)의 안전한 작업방법 및 추락, 낙하, 붕괴, 감전
등의 재해를 예방하기 위하여 필요한 작업 단계별 안전사항 및 안전시설에 관
한 기술적 사항 등을 정함을 목적으로 한다.
2. 적용범위
이 지침은 바닥으로
----------------------------------------
📄 Source: I.L.M 교량공사 안전보건작업 지침.pdf
I.L.M 교량공사 안전보건작업 지침
1. 목 적
이 지침은 산업안전보건기준에 관한 규칙(이하 “안전보건규칙”이라 한다.) 제
42조 내지 제49조의 규정에 의거 콘크리트 구조물 교량공사(I.L.M공법)의
안전한 작업방법 및 추락, 낙하, 붕괴, 감전 등의 재해를 예방하기 위하여
필요한 작업 단계별 안전사항 및 안전시설에 관한 기술적 사항 등을 정함을
목적으로 한다.
2. 적용범위
이 지침은 콘크리트 구조물 교량공사에서 상부구조의 가설공법 중 교대 후방에
설치한 주형 제작장에서 1세그먼트씩 제작하여 프리스트레스에 의해 일체화한
후 
----------------------------------------
📄 Source: PCT거더 교량공사 안전보건작업지침.pdf
KOSHA GUIDE
C - 83 - 2013
PCT거더 교량공사 안전보건작업
지침
2013. 9.
한국산업안전보건공단

안전보건작업지침의 개요
○ 작성자 : 한국안전학회 백신원
○ 제·개정 경과
- 2013년 9월 건설안전분야 제정위원회 심의(제정)
○ 관련규격 및 자료
- 최신 교량공학(동명사, 황학주)
- 도로교 표준시방서(대한토목학회)
- 교량공사(PCT거더공법) 시공자

In [59]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

# 텍스트 스플리터 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # 각 청크의 최대 길이
    chunk_overlap=100,  # 청크 간 겹치는 부분
    separators=['\n\n', '\n', '.', ' ']  # 텍스트 분할 기준
)

# 모든 문서를 청크 단위로 분할
split_documents = []
for doc in processed_documents:
    chunks = text_splitter.split_text(doc.page_content)  # 문서의 내용 분할
    
    # 분할된 청크를 새로운 Document 리스트에 저장
    for chunk in chunks:
        split_documents.append(Document(page_content=chunk, metadata=doc.metadata))

# 결과 확인 (첫 3개만 출력)
for doc in split_documents[:1]:
    print(doc.metadata, doc.page_content)

{'source': 'F.C.M 교량공사 안전보건작업 지침.pdf'} F.C.M 교량공사 안전보건작업 지침
1. 목 적
이 지침은 산업안전보건기준에 관한 규칙(이하 “안전보건규칙”이라 한다.) 제
42조~제49조(추락에 의한 위험방지), 제133조~제150조(크레인, 이동식 크레인),
제369조(교량작업), 제328조~제337조(거푸집동바리 등)의 규정에 의거 콘크리
트 구조물 교량공사(F.C.M공법)의 안전한 작업방법 및 추락, 낙하, 붕괴, 감전
등의 재해를 예방하기 위하여 필요한 작업 단계별 안전사항 및 안전시설에 관
한 기술적 사항 등을 정함을 목적으로 한다.
2. 적용범위
이 지침은 바닥으로부터 동바리를 사용하지 않고 교각위 주두부(Pier table)를
시공한 후 교각 양쪽의 교축방향으로 특수한 가설장비(F/T, Form traveller)를
이용해 좌․우로 하중의 균형을 맞추면서 세그먼트(Segment)의 콘크리트 타설,
프리스트레싱 도입을 순차적으로 반복하여 교량 상부 구조를 완성하는 현장타
설 F.C.M 공법에 적용한다.
<그림 1> F.C.M 교량시공 전경
- 1 -


In [ ]:
def page_split(content):
    '''page별로 쪼개버림'''
    page_dict = {}
    i = 0
    while True:
        i += 1
        contents = content.split(f"- {i} -")
        if len(contents) != 2:
            break
        page_dict[str(i)] = contents[0]
        content = contents[1]
    return page_dict

page_split(all_documents[0].page_content)

In [55]:
import re
import os
from langchain_core.documents import Document

# 섹션 제목을 추출하는 함수
def extract_section(text):
    """
    문서 내에서 '1. 제목'과 같은 형식의 섹션을 찾는 함수.
    가장 먼저 등장하는 섹션을 반환하며, 없으면 'Unknown' 반환.
    """
    pattern = r"^\s*(\d+)\.\s+(.+)"  # '1. 제목' 또는 ' 1. 제목' 같은 패턴 탐색
    for line in text.split("\n"):  # 줄 단위로 검색
        match = re.match(pattern, line)
        if match:
            return match.group(2).strip()  # 숫자 다음에 오는 제목 부분만 반환
    return "Unknown"  # 해당하는 패턴이 없을 경우 기본값 반환

# 메타데이터 변환 함수
def custom_metadata(doc):
    metadata = doc.metadata  # 기존 메타데이터 가져오기
    text = doc.page_content  # 문서의 실제 내용 가져오기
    
    new_metadata = {
        "title": os.path.basename(metadata.get("source", "Unknown")),  # 파일 이름만 저장
        "page": metadata.get("page", 0) + 1,  # ✅ 페이지 번호를 1부터 시작하도록 수정
        "section": extract_section(text)  # ✅ 문서 내용에서 섹션 제목 추출
    }
    
    doc.metadata = new_metadata  # 기존 문서의 메타데이터 수정
    return doc

# 모든 문서의 메타데이터 변환 적용
documents = [custom_metadata(doc) for doc in split_documents]

# 결과 확인 (2개 출력)
for doc in documents[0:]:  
    print(doc.metadata)

{'title': 'F.C.M 교량공사 안전보건작업 지침.pdf', 'page': 1, 'section': '목 적'}
{'title': 'F.C.M 교량공사 안전보건작업 지침.pdf', 'page': 1, 'section': '용어의 정의'}
{'title': 'F.C.M 교량공사 안전보건작업 지침.pdf', 'page': 1, 'section': 'Unknown'}
{'title': 'F.C.M 교량공사 안전보건작업 지침.pdf', 'page': 1, 'section': 'Unknown'}
{'title': 'F.C.M 교량공사 안전보건작업 지침.pdf', 'page': 1, 'section': 'F.C.M공법 작업절차'}
{'title': 'F.C.M 교량공사 안전보건작업 지침.pdf', 'page': 1, 'section': '공통적인 안전조치사항'}
{'title': 'F.C.M 교량공사 안전보건작업 지침.pdf', 'page': 1, 'section': 'Unknown'}
{'title': 'F.C.M 교량공사 안전보건작업 지침.pdf', 'page': 1, 'section': '주두부 안전작업'}
{'title': 'F.C.M 교량공사 안전보건작업 지침.pdf', 'page': 1, 'section': 'Unknown'}
{'title': 'F.C.M 교량공사 안전보건작업 지침.pdf', 'page': 1, 'section': '이동식 작업대차 안전작업'}
{'title': 'F.C.M 교량공사 안전보건작업 지침.pdf', 'page': 1, 'section': 'Unknown'}
{'title': 'F.C.M 교량공사 안전보건작업 지침.pdf', 'page': 1, 'section': 'Unknown'}
{'title': 'F.C.M 교량공사 안전보건작업 지침.pdf', 'page': 1, 'section': '세그먼트(Segment) 및 키 세그먼트(Key segment) 시공시 안전작업'}
{'title': 'F.C.M 교량공사 안전보건작업 